In [1]:
!pip install -r requirements.txt

In [2]:
import openai
import gradio as gr
import os


In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [29]:
def gpt3(query,database):
    response = openai.Completion.create(
    model="text-ada-001",
    prompt="You are a MongoDB query generator for a database with the following collections and their properties:\n\n\n" + database + "\n\nUser: List the names of the departments which employed more than 10 employees in the last 3 months\nAssistant:\n'''\ndb.Department.aggregate([\n  {\n    $lookup: {\n      from: \"Employee\",\n      localField: \"_id\",\n      foreignField: \"department_id\",\n      as: \"employees\"\n    }\n  },\n  {\n    $lookup: {\n      from: \"Salary_Payments\",\n      localField: \"employees._id\",\n      foreignField: \"employee_id\",\n      as: \"salary_payments\"\n    }\n  },\n  {\n    $match: {\n      \"salary_payments.date\": { $gte: new Date(new Date().getTime() - 1000 * 60 * 60 * 24 * 90) }\n    }\n  },\n  {\n    $group: {\n      _id: \"$name\",\n      employeeCount: { $sum: 1 }\n    }\n  },\n  {\n    $match: {\n      employeeCount: { $gt: 10 }\n    }\n  },\n  {\n    $project: {\n      _id: 0,\n      department_name: \"$_id\"\n    }\n  }\n]);\n'''\nUser: List the names of the departments which employed more than 3 employees and has id less than 3400\nAssistant: \n'''\ndb.Department.aggregate([\n  {\n    $match: { _id: { $lt: 3400 } }\n  },\n  {\n    $lookup: {\n      from: \"Employee\",\n      localField: \"_id\",\n      foreignField: \"department_id\",\n      as: \"employees\"\n    }\n  },\n  {\n    $group: {\n      _id: \"$name\",\n      employeeCount: { $sum: { $size: \"$employees\" } }\n    }\n  },\n  {\n    $match: {\n      employeeCount: { $gt: 3 }\n    }\n  },\n  {\n    $project: {\n      _id: 0,\n      department_name: \"$_id\"\n    }\n  }\n]);\n'''\nUser: " + query + "\nAssistant:\n'''\n",
    temperature=0,
    max_tokens=250,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["User:"]
    )

    x = response.choices[0].text
    
    return x 

In [30]:
def greet(query,database):
    sql = gpt3(query,database)
    return sql

In [31]:
greet("List the names of the departments which employed less than 10 employees","Employee(id, name, department_id)\nDepartment(id, name, address)\nSalary_Payments(id, employee_id, amount, date)")

'db.Department.aggregate([\n\n{\n    $lookup: {\n      from: "Employee",\n      localField: "_id",\n      foreignField: "department_id",\n      as: "employees"\n    }\n   },\n  {\n    $lookup: {\n      from: "Salary_Payments",\n      localField: "employees._id",\n      foreignField: "employee_id",\n      as: "salary_payments"\n    },\n  {\n    $match: {\n      "salary_payments.date": { $gte: new Date(new Date().getTime() - 1000 * 60 * 60 * 24 * 90) }\n    }\n  },\n  {\n    $match: {\n      "salary_payments.date": { $gt: 10 }\n '

In [33]:
with gr.Blocks() as demo:
    textbox = gr.Textbox(placeholder="Type here...", lines=10)
    btn = gr.Button("Complete")
    
    # define what will run when the button is clicked, here the textbox is used as both an input and an output
    btn.click(fn=greet, inputs=textbox, outputs=textbox, queue=False)

In [34]:
examples = [
    ["List the names of the departments which employed more than 10 employees","Employee(_id, name, department_id)\nDepartment(_id, name, address)\nSalary_Payments(_id, employee_id, amount, date)"],
    ["List the names of the departments which employed less than 10 employees","Employee(_id, name, department_id)\nDepartment(_id, name, address)\nSalary_Payments(_id, employee_id, amount, date)",]
] 

In [35]:
demo = gr.Interface(
    fn=gpt3,
    inputs=[
        gr.components.Textbox(lines=5, label="Natural Language Query"),
        gr.components.Textbox(lines=5, label="Database Format"),
    ],
    outputs=gr.components.Textbox(label="NoSQL Output"),
    examples=examples
) 

In [36]:
demo.launch(share=False, debug=True) 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
